In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Librerias

In [ ]:
import re, string, unicodedata
import numpy as np
import pandas as pd
import pickle
import collections

import nltk
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords


%pip install stanza
import stanza


In [ ]:
download('stopwords')
download('punkt')
stanza.download('es')
nlp = stanza.Pipeline('es')

In [ ]:
def eliminar_textos_basura(texto):
    # eliminar emojis: elimina todo lo que este en <>. 
    texto_procesado = re.sub('<.*?>', ' ', texto)
    # convertir a minusculas
    texto_procesado = texto_procesado.lower()
    # remover @usuario
    texto_procesado = re.sub('@[^\s]+',' ',texto_procesado)
    # remover RT
    texto_procesado = re.sub('rt   ',' ',texto_procesado)
    texto_procesado = re.sub('rt',' ',texto_procesado)
    texto_procesado = re.sub('rt ',' ',texto_procesado)
    # remover numeros
    texto_procesado = re.sub("\d+", "", texto_procesado)
    # texto_procesado = re.sub(r'http\S+', ' ', texto_procesado)
    texto_procesado = re.sub("(\w+:\/\/\S+)", " ", texto_procesado)
    # reemplazar todos los caracteres que no saen alfanuméricos con espacios
    texto_procesado = re.sub(r'[^a-zA-Z0-9ÑñÁáÉéÍíÓóÚú\s]', ' ', texto_procesado)
    
    return texto_procesado

def eliminar_stop_words(texto):
    # obtener lista de stopwords
    stop_words = set(stopwords.words('spanish')) 
    stop_words.remove('no')
    #separar el texto por palabras
    palabras = word_tokenize(texto) 
    # dejar solo palabras que no sean stopwords
    texto_sin_sw = [w for w in palabras if not w in stop_words] 
    #  convertir en una sola cadena la lista de palabras
    texto_sin_sw = ' '.join(texto_sin_sw)

    return texto_sin_sw 

def remove_non_ascii(word):
    """Remueve caracteres no ASCII"""
    new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    return new_word

def establecer_lema(text):
  doc = nlp(text)
  new = ''
  for sent in doc.sentences:
    for word in sent.words:
      new = new + word.lemma
      new = new + ' '

  return new

#Convierto cada tweet del dataset en un vector de palabras
from nltk import word_tokenize
def ConvertirData(corpusBase):
  textos = []
  for y in [corpusBase['text'][i] for i in range(len(corpusBase))]:
      doc = word_tokenize(y)
      textos.append(doc)
  return textos

In [ ]:
def Knn(Matriz, k):
  labels = DataFinal['tipo']
  pred_label = []
  etiquetar = []
  indice = 0
  for x in labels:
      if(indice != len(labels)-1):
        if(Matriz[indice]<=0.20): #Filtro que asigan directament 0 cuando el valor de comparacion sera cero o menor a cero
          etiquetar.append((Matriz[indice], 0))
        else:
          etiquetar.append((Matriz[indice], x))
      indice += 1
  etiquetar.sort(reverse=True)
  neighbors = etiquetar[:k]
  votes = []
  for neighbor in neighbors:
            votes.append(neighbor[1])
  counter = collections.Counter(votes) #Determina el que mas se repite
  pred_label.append(counter.most_common()[0][0])
  return pred_label

def EtiquetaEnTexto(valor):
  if valor[0]==1:
    return 'emergencia'
  else:
    return 'no emergencia'

In [ ]:
def ProcesarTweet(Tweet):
  Procesado = eliminar_textos_basura(Tweet)
  Procesado = eliminar_stop_words(Procesado)
  Procesado = remove_non_ascii(Procesado)
  Procesado = establecer_lema(Procesado)
  return Procesado

RECURSO

In [ ]:
DataFinal = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Reporte_tweets_Ecuador/4DataFinalLemaFiltro.xlsx')

CREACIÓN DEL MODELO 


In [ ]:
from gensim import corpora
from gensim.models import LsiModel
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity

In [ ]:
'''
Función que recibira el tweet para generar las comparaciones con los tweets de la data en base al modelo
'''
def modeloLSI(tweet):
  Tweet = word_tokenize(ProcesarTweet(tweet)) #Se Preprocesa el tweet y se tokeniza el texto
  
  if len(Tweet) <= 2: #Se valida que el tweet preprocesado tenga un minino de palabras
    Tweet = ['vacio']  #Se le asigna el texto 'vacio' el cual sera detectado como no emergencia
  
  tweet_Dic = DiccioLSI.doc2bow(Tweet) #Se estructura el tweet en función al diccionario del modelo
  sim = MatrizSimLSI[modelLSI[tweet_Dic]] #Se genera un nuevo vector que contiene el nivel de similitud del nuevo tweet con cada tweet del dataset
  
  Valor_Similitud = [] 
  for i in range(len(sim)):
    a = ('%.2f' % sim[i]) #Se redondea cada valor del vector de similitud creado para establecer un mismo formato a todos los valores
    Valor_Similitud.append(float(a))
  
  return Valor_Similitud

In [ ]:
'''
Determina la etiqueta del nuevo tweet por cada knn establecido
'''
def EtiquetarModelLSI(tweet):
  Similaridad= modeloLSI(tweet)
  k_1 = Knn(Similaridad, 1)
  k_3 = Knn(Similaridad, 3)
  k_5 = Knn(Similaridad, 5)
  k_7 = Knn(Similaridad, 7)
  k_9 = Knn(Similaridad, 9)
  k_11 = Knn(Similaridad, 11)
  return k_1, k_3, k_5, k_7, k_9, k_11 

In [ ]:
'''
En consideración a que el modelo cada que se crea genera resultados diferentes, se establece la creación de 12 variantes 
'''
textos = ConvertirData(DataFinal) #Se tokeniza cada tweet de la data
for a in range(12):
  #Creación del modelo
  dictionary_textos = corpora.Dictionary(textos) #Se genera el diccionario en base a la data
  corpus_gensim_textos = [dictionary_textos.doc2bow(doc) for doc in textos]
  tfidf_textos = TfidfModel(corpus_gensim_textos) #Se crea el modelo de frecuencia de palabras  
  corpus_tfidf_textos = tfidf_textos[corpus_gensim_textos] #Se genera una nueva corpus en base al modelo creado
  lsi_textos = LsiModel(corpus_tfidf_textos, id2word=dictionary_textos, num_topics=700) #Se crea el modelo  LSI usando la corpus que se genero
  lsi_index_textos= MatrixSimilarity(lsi_textos[corpus_tfidf_textos]) #Se establece la matriz de similaridad

  text1= '/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/DiccionarioLSI'+str(a)+'.pickle'
  text2= '/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/MatrizSimilaridadLSI'+str(a)+'.pickle'
  text3= '/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/ModelLSI'+str(a)+'.model'

  pickle.dump(dictionary_textos, open(text1, 'wb'))
  pickle.dump(lsi_index_textos, open(text2, 'wb'))
  pickle.dump(lsi_textos, open(text3, 'wb'))

  print('MODELO CREADO LSI '+str(a))

MODELO CREADO LSI 0
MODELO CREADO LSI 1
MODELO CREADO LSI 2
MODELO CREADO LSI 3
MODELO CREADO LSI 4
MODELO CREADO LSI 5
MODELO CREADO LSI 6
MODELO CREADO LSI 7
MODELO CREADO LSI 8
MODELO CREADO LSI 9
MODELO CREADO LSI 10
MODELO CREADO LSI 11


PRUEBAS DE LAS VARIANTES DEL MODELO

In [ ]:
import pandas as pd
import numpy as np
import collections
from sklearn import datasets, metrics

def EmplearMetricasDeValidacion(DataPrueba, Modelo, columnas):
  metric1 = []
  metric2 = []
  metric3 = []
  metric4 = []
  metric5 = []

  re = GenerarResultadosModelo(DataPrueba, Modelo)
  #k1
  metric1.append(metrics.accuracy_score(re[0], re[1]))
  metric2.append(metrics.precision_score(re[0], re[1]))
  metric3.append(metrics.recall_score(re[0], re[1]))
  #metric4.append(metrics.f1_score(re[0], re[1]))
  metric5.append(metrics.log_loss(re[0], re[1]))
  #k3
  metric1.append(metrics.accuracy_score(re[0], re[2]))
  metric2.append(metrics.precision_score(re[0], re[2]))
  metric3.append(metrics.recall_score(re[0], re[2]))
  #metric4.append(metrics.f1_score(re[0], re[2]))
  metric5.append(metrics.log_loss(re[0], re[2]))
  #k5
  metric1.append(metrics.accuracy_score(re[0], re[3]))
  metric2.append(metrics.precision_score(re[0], re[3]))
  metric3.append(metrics.recall_score(re[0], re[3]))
  #metric4.append(metrics.f1_score(re[0], re[3]))
  metric5.append(metrics.log_loss(re[0], re[3]))
  #k7
  metric1.append(metrics.accuracy_score(re[0], re[4]))
  metric2.append(metrics.precision_score(re[0], re[4]))
  metric3.append(metrics.recall_score(re[0], re[4]))
  #metric4.append(metrics.f1_score(re[0], re[4]))
  metric5.append(metrics.log_loss(re[0], re[4]))

  #k9
  metric1.append(metrics.accuracy_score(re[0], re[5]))
  metric2.append(metrics.precision_score(re[0], re[5]))
  metric3.append(metrics.recall_score(re[0], re[5]))
  #metric4.append(metrics.f1_score(re[0], re[5]))
  metric5.append(metrics.log_loss(re[0], re[5]))

  #k11
  metric1.append(metrics.accuracy_score(re[0], re[6]))
  metric2.append(metrics.precision_score(re[0], re[6]))
  metric3.append(metrics.recall_score(re[0], re[6]))
  #metric4.append(metrics.f1_score(re[0], re[6]))
  metric5.append(metrics.log_loss(re[0], re[6]))

  resultado=[metric1,metric2,metric3,metric5]

  Final = pd.DataFrame(resultado,columns=columnas)
  Final.insert(0,"METRICAS VALIDACION",['ACURRACY_SCORE','PRECISIÓN_SCORE','RECALL_SCORE','LOG_LOSS'],True)
  
  return Final

def GenerarResultadosModelo(DataN, modelo):
  Data = DataN
  Reemplazo = Data['tipo'].replace(['emergencia'], 1).replace(['no emergencia'], 0)
  Data['tipo'] = Reemplazo
  text = list(Data['New_Tweet'])
  
  prediccion1 = []
  prediccion2 = []
  prediccion3 = []
  prediccion4 = []
  prediccion5 = []
  prediccion6 = []

  for a in text:
    metodo= modelo(a)
    prediccion1.append(metodo[0])
    prediccion2.append(metodo[1])
    prediccion3.append(metodo[2])
    prediccion4.append(metodo[3])
    prediccion5.append(metodo[4])
    prediccion6.append(metodo[5])
  test_final = list(Data['tipo'])

  return  test_final, prediccion1, prediccion2, prediccion3, prediccion4, prediccion5, prediccion6 

In [ ]:
#Generar Reportes que contengan los resultados de las metricas de validación de cada una de las variantes del modelo
import pickle
DataNivelada = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Tweets_Etiquetados/New_DataPrueba_Total.xlsx')

Data = []

for a in range(12):
  
  txt1 = '/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/ModelLSI'+str(a)+'.model'
  txt2 = '/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/DiccionarioLSI'+str(a)+'.pickle'
  txt3 = '/content/gdrive/My Drive/Colab Notebooks/MODELOLSI/Todos5/MatrizSimilaridadLSI'+str(a)+'.pickle'

  modelLSI = pickle.load(open(txt1,'rb'))
  DiccioLSI = pickle.load(open(txt2,'rb'))
  MatrizSimLSI = pickle.load(open(txt3,'rb'))

  LSI = EmplearMetricasDeValidacion(DataNivelada, EtiquetarModelLSI, ['lsi_1','lsi_3','lsi_5','lsi_7','lsi_9','lsi_11'])
  txtData = 'Verificacion_Modelo_LSI_'+str(a)+'.xlsx'
  print('ENTRO: ',a)
  Data.append(LSI)

In [ ]:
#Guardar Reportes, el cual contara con las metricas de validación de cada uno de las variantes del modelo
Data[0].to_excel('Verificacion_Modelo_LSI_0.xlsx', index=False)
!cp Verificacion_Modelo_LSI_0.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[1].to_excel('Verificacion_Modelo_LSI_1.xlsx', index=False)
!cp Verificacion_Modelo_LSI_1.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[2].to_excel('Verificacion_Modelo_LSI_2.xlsx', index=False)
!cp Verificacion_Modelo_LSI_2.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[3].to_excel('Verificacion_Modelo_LSI_3.xlsx', index=False)
!cp Verificacion_Modelo_LSI_3.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[4].to_excel('Verificacion_Modelo_LSI_4.xlsx', index=False)
!cp Verificacion_Modelo_LSI_4.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[5].to_excel('Verificacion_Modelo_LSI_5.xlsx', index=False)
!cp Verificacion_Modelo_LSI_5.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[6].to_excel('Verificacion_Modelo_LSI_6.xlsx', index=False)
!cp Verificacion_Modelo_LSI_6.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[7].to_excel('Verificacion_Modelo_LSI_7.xlsx', index=False)
!cp Verificacion_Modelo_LSI_7.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[8].to_excel('Verificacion_Modelo_LSI_8.xlsx', index=False)
!cp Verificacion_Modelo_LSI_8.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[9].to_excel('Verificacion_Modelo_LSI_9.xlsx', index=False)
!cp Verificacion_Modelo_LSI_9.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[10].to_excel('Verificacion_Modelo_LSI_10.xlsx', index=False)
!cp Verificacion_Modelo_LSI_10.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'

Data[11].to_excel('Verificacion_Modelo_LSI_11.xlsx', index=False)
!cp Verificacion_Modelo_LSI_11.xlsx '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'


Análisis de Resultados para determinar la mejor variante del Modelo




In [ ]:
#Cargar los resultados de validación de las variantes del modelo
DataModel = []
for a in range(12): 
  txt = 'Verificacion_Modelo_LSI_'+str(a)+'.xlsx'
  txt2= '/content/gdrive/My Drive/Colab Notebooks/ANALIZAR/Todo6/'+txt
  DataModel.append(pd.read_excel(txt2)) 

In [ ]:
Lsi_5 = []
Lsi_7 = []
Lsi_9 = []
Lsi_11 = []
for a in range(9):
  Lsi_5.append([DataModel[a]['lsi_5'][0],a])
  Lsi_7.append([DataModel[a]['lsi_7'][0],a])
  Lsi_9.append([DataModel[a]['lsi_9'][0],a])
  Lsi_11.append([DataModel[a]['lsi_11'][0],a])


In [ ]:
Lsi_5.sort(reverse=True)
Lsi_5

[[0.9, 6],
 [0.89875, 1],
 [0.8975, 7],
 [0.89625, 8],
 [0.8925, 5],
 [0.8925, 3],
 [0.89125, 4],
 [0.89, 2],
 [0.89, 0]]

In [ ]:
Lsi_7.sort(reverse=True)
Lsi_7

[[0.905, 0],
 [0.9025, 8],
 [0.9025, 3],
 [0.89875, 7],
 [0.89875, 5],
 [0.8975, 1],
 [0.895, 6],
 [0.895, 2],
 [0.89375, 4]]

In [ ]:
Lsi_9.sort(reverse=True)
Lsi_9

[[0.89625, 3],
 [0.895, 6],
 [0.89375, 0],
 [0.89125, 8],
 [0.89125, 2],
 [0.89, 4],
 [0.88875, 1],
 [0.8875, 7],
 [0.885, 5]]

In [ ]:
Lsi_11.sort(reverse=True)
Lsi_11

[[0.89375, 0],
 [0.88875, 6],
 [0.88875, 5],
 [0.88875, 4],
 [0.88875, 2],
 [0.88625, 8],
 [0.88625, 3],
 [0.8825, 7],
 [0.88125, 1]]

In [ ]:
'''
Por lo tanto la mejor variante del modelo es la primera (0), con un K=7
'''
DataModel[0]

,METRICAS VALIDACION,lsi_1,lsi_3,lsi_5,lsi_7,lsi_9,lsi_11
0,ACURRACY_SCORE,0.818750,0.870000,0.890000,0.905000,0.893750,0.893750
1,PRECISIÓN_SCORE,0.792079,0.852417,0.870886,0.890026,0.873737,0.877551
2,RECALL_SCORE,0.839895,0.879265,0.902887,0.913386,0.908136,0.902887
3,LOG_LOSS,6.260237,4.490099,3.799316,3.281227,3.669795,3.669793
